# Assignment 2 

## Kun Huang, Yizhou Jin

In this assignment, we try some features in **TensorFlow** and **gym**. We tried to rewrite the example in a tensorflow style, but since it involves **gym**, we have not figured out how to rewrite it completely.   

First we import all the libraries necessary. 

In [3]:
import gym
import numpy as np
import random
import tensorflow as tf
from IPython.display import clear_output
import time
import warnings

warnings.filterwarnings('ignore')

In [4]:
#Set environment in gym
env = gym.make('FrozenLake-v0')

## Codes 1

The idea mainly comes from [Simple Reinforcement Learning with Tensorflow Part 0: Q-Learning with Tables and Neural Networks](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0). It uses the idea of neural networks, that is, it uses a one-layer network which takes the state encoded in a vector (1x16,i.e. `inputs1`), and produces a vector of 4 Q-values (i.e. `Qout` and `nextQ`). 

In [5]:
tf.reset_default_graph()

#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.1))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

# loss, L2
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

Next, we run the codes rewritten in tensorflow.

In [7]:
# set a initializer for tensor
init = tf.global_variables_initializer()

# Set learning parameters
gamma = .7
episodes = 5000
alpha = 0.2

total = 0

with tf.Session() as sess:
    sess.run(init)
    for i in range(episodes):
        
        state = env.reset()
        done = False
        epsilon = 1./(i+1)
        # first act
        action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
        targetQ = allQ
        #The Q-Network
        while not done:
            
            #Choose an action
            if random.uniform(0,1) < epsilon:
                action[0] = env.action_space.sample()
            else:
                action[0] = int(np.argmax(targetQ))
            #Get new state and reward from env
            new_state, reward, done, info = env.step(action[0])
            
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout, feed_dict={inputs1:np.identity(16)[new_state:new_state + 1]})
            #np.max(Q[new_state])
            maxQ1 = np.max(Q1)
            # formula
            targetQ[0,action[0]] += alpha * (reward + gamma*maxQ1 - targetQ[0, action[0]])
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[state:state+1],nextQ:targetQ})
            total += reward
            state = new_state
            
print ("Average rewards: " + str(total/episodes))
print("Learned Q-Table")
print(W1)


Average rewards: 0.0488
Learned Q-Table
[[0.00210785 0.00403071 0.00237279 0.00221303]
 [0.00210785 0.00620063 0.00237279 0.00221303]
 [0.00210785 0.00456139 0.00237279 0.00221303]
 [0.00210785 0.00553866 0.00237279 0.00221303]
 [0.00210785 0.01012959 0.00237279 0.00221303]
 [0.01619693 0.01145797 0.07632474 0.0677774 ]
 [0.00210785 0.01378301 0.00237279 0.00221303]
 [0.06326784 0.08481377 0.04293985 0.09981561]
 [0.00210785 0.00497512 0.00237279 0.00221303]
 [0.00210785 0.00516598 0.00237279 0.00221303]
 [0.00210785 0.01198854 0.00237279 0.00221303]
 [0.0110678  0.05297069 0.07216506 0.00177388]
 [0.0997953  0.08007695 0.02440802 0.03464976]
 [0.00210785 0.01560906 0.00237279 0.00221303]
 [0.00210785 0.03036444 0.00237279 0.00221303]
 [0.00559726 0.03219075 0.06012447 0.01850482]]


## Code2

Similarly, we rewrite the sample code into tensorflow.

In [8]:
env.reset()
episodes = 10
max_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for ep in range(episodes):
        state = env.reset()
        step = 0
        done = False
        clear_output()
        print("*****************")
        print("Episodes", ep)
        print("*****************")
        for step in range(max_steps):
            action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
            action[0] = np.argmax(allQ)
            new_state, reward, done, info = env.step(action[0])
            if done:
                env.render()
                print("*****************")
                print("Steps", step)
                print("*****************")
                time.sleep(2)
                break
        state = new_state
env.close()


*****************
Episodes 9
*****************
  (Right)
SFFF
FHFH
FFFH
HFFG
*****************
Steps 2
*****************
